# CLUE

In [ ]:
#| default_exp methods.vaecf

In [ ]:
#| include: false
%load_ext autoreload
%autoreload 2
from ipynb_path import *
from nbdev import show_doc
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#| export
from __future__ import annotations
from relax.import_essentials import *
from relax.methods.base import BaseCFModule, BaseParametricCFModule
from relax.utils import *
from relax.module import MLP, BaseTrainingModule
from relax.data import *
from relax.trainer import train_model, TrainingConfigs

In [ ]:
#| export
class Encoder(hk.Module):
    def __init__(self, sizes: List[int], dropout: float = 0.1):
        super().__init__()
        assert sizes[-1] % 2 == 0, f"sizes[-1] must be even, but got {sizes[-1]}"
        self.encoder = MLP(
            sizes, dropout_rate=dropout, name="encoder_mean")
    
    def __call__(self, x: Array, is_training: bool):
        params = self.encoder(x, is_training)
        d = params.shape[-1] // 2
        mu, sigma = params[:, :d], params[:, d:]
        sigma = jax.nn.softplus(sigma)
        sigma = jnp.clip(sigma, 1e-3)
        return mu, sigma

class Decoder(hk.Module):
    def __init__(
        self, 
        sizes: List[int], 
        input_size: int,
        dropout: float = 0.1
    ):
        super().__init__()
        self.decoder = MLP(
            sizes, dropout_rate=dropout, name="Decoder")
        self.input_size = input_size
    
    def __call__(self, z: Array, is_training: bool):
        mu_dec = self.decoder(z, is_training=is_training)
        mu_dec = hk.Linear(self.input_size, name='mu_x')(mu_dec)
        mu_dec = jax.nn.sigmoid(mu_dec)
        return mu_dec


In [ ]:
#| export
class VAEGaussCat(BaseTrainingModule):
    def __init__(self, m_configs: Dict = None):
        if m_configs is None: m_configs = {}
        self.save_hyperparameters(m_configs)
        self.m_config = validate_configs(m_configs, VAECFModuleConfigs)
        self.opt = optax.radam(self.m_config.lr)

    def _update_categorical_info(self):
        cat_arrays = self._data_module._cat_arrays
        self._cat_info = {
            'cat_idx': self._data_module.cat_idx,
            'cat_arr': [len(cat_arr) for cat_arr in cat_arrays],
        }
    
    def init_net_opt(self, dm, key):
        self._data_module = dm
        self._update_categorical_info()
        keys = jax.random.split(key, 3)
        X, y = dm.train_dataset[:128]
        Z = jnp.ones((X.shape[0], self.m_config.enc_sizes[-1]))
        inputs = jnp.concatenate([X, y.reshape(-1, 1)], axis=-1)
        latent = jnp.concatenate([Z, y.reshape(-1, 1)], axis=-1)

        self.encoder = make_hk_module(
            Encoder, sizes=self.m_config.enc_sizes, 
            dropout=self.m_config.dropout_rate
        )
        self.decoder = make_hk_module(
            Decoder, sizes=self.m_config.dec_sizes,
            input_size=X.shape[-1], 
            dropout=self.m_config.dropout_rate
        )

        enc_params = self.encoder.init(
            keys[0], inputs, is_training=True)
        dec_params = self.decoder.init(
            key[1], latent, is_training=True)
        opt_state = self.opt.init((enc_params, dec_params))

        # set prior for training latents
        self.prior = jrand.normal(
            keys[2], (self.m_config.enc_sizes[-1],)
        )
        return (enc_params, dec_params), opt_state
    
    @partial(jax.jit, static_argnums=(0, 4))
    def encode(self, enc_params, rng_key, x, is_training=True):
        mu_z, var_z = self.encoder.apply(
            enc_params, rng_key, x, is_training=is_training)
        return mu_z, var_z
    
    @partial(jax.jit, static_argnums=(0, ))
    def sample_latent(self, rng_key, mean, var):
        eps = jax.random.normal(rng_key, var.shape)
        return mean + eps * var
    
    @partial(jax.jit, static_argnums=(0, 4))
    def decode(self, dec_params, rng_key, z, is_training=True):
        reconstruct_x = self.decoder.apply(
            dec_params, rng_key, z, is_training=is_training)
        return reconstruct_x
    
    @partial(jax.jit, static_argnums=(0, 5))
    def sample_step(
        self, rng_key, dec_params, mean, var, is_training=True
    ):
        z = self.sample_latent(rng_key, mean, var)
        mu_x = self.decode(dec_params, rng_key, z, is_training=is_training)
        return mu_x
    
    @partial(jax.jit, static_argnums=(0, 4, 5))
    def sample(
        self, params, rng_key, x, mc_samples, is_training=True
    ): # Shape: (mc_samples, batch_size, input_size)
        enc_params, dec_params = params
        mean, var = self.encode(enc_params, rng_key, x, is_training=is_training)
        keys = jax.random.split(rng_key, mc_samples)
        
        partial_sample_step = partial(
            self.sample_step, dec_params=dec_params,
            mean=mean, var=var, is_training=is_training
        )
        reconstruct_x = jax.vmap(partial_sample_step)(keys)
        return (mean, var, reconstruct_x)
    
    @partial(jax.jit, static_argnums=(0, 4))
    def compute_loss(self, params, rng_key, x, is_training=True):
        @partial(jax.jit, static_argnums=(2, 3))
        def reconstruct_loss(x: Array, cf: Array, cat_idx: int, cat_arr: List[int]):
            cont_loss = optax.l2_loss(x[:, :cat_idx], cf[:, :cat_idx])
            cat_loss = optax.softmax_cross_entropy(
                cf[:, cat_idx:], x[:, cat_idx:])
            return jnp.concatenate([cont_loss, cat_loss], axis=-1).sum(-1)
        
        


In [ ]:
optax.softmax_cross_entropy(
    jnp.array([[0.1, 0.2, 0.7], [0.7, 0.2, 0.1]]),
    jnp.array([[0., 0., 1.], [1., 0, 0]])
)

DeviceArray([0.7679496, 0.7679496], dtype=float32)